## This notebook is an example of how to pipeline two models. 
A video stream from a local camera is processed by the license plate detection model. 
The detection results are then processed by license number recognition model, 
one bounding box at a time. Combined result is then displayed.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](../../env.ini) file, located in the same directory as this notebook.

**Access to camera is required to run this sample.**

The script needs either a web camera or local camera connected to the machine running this code. The camera index or URL needs to be specified either in the code below by assigning `camera_id` or in [env.ini](../../env.ini) file by defining `CAMERA_ID` variable and assigning `camera_id = None`.

#### Specify camera index 

In [ ]:
camera_id = None  # camera index or URL; 0 to use default local camera, None to take from env.ini file

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, dgtools

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD  # <-- on the Cloud Platform
# target = dgtools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, dgtools.get_cloud_zoo_url(), dgtools.get_token())

#### The rest of the cells below should run without any modifications

In [ ]:
# load models for DeGirum Orca AI accelerator
lp_det_model = zoo.load_model("yolo_v5s_lp_det--512x512_quant_n2x_orca1_1")
lp_ocr_model = zoo.load_model("yolo_v5s_lp_ocr--256x256_quant_n2x_orca1_1")

# adjust some model properties
lp_det_model.overlay_alpha = 1
lp_det_model.overlay_line_width = 1

In [ ]:
import queue


# Specialized non-blocking queue which acts as iterator
class BoxQueue(queue.Queue):
    def __iter__(self):
        while True:
            try:
                value = self.get_nowait()
                if value is None:
                    break  # `None` sentinel signals end of queue
                yield value
            except queue.Empty:
                yield None  # in case of empty queue, yield None

In [ ]:
# AI prediction loop, press 'x' or 'q' to stop video
with dgtools.Display("AI Camera") as display:

    lp_queue = BoxQueue()  # queue for license plate boxes
    lp_ocr_model.non_blocking_batch_predict = True  # use non-blocking mode for nested model
    lp_ocr_model_iter = lp_ocr_model.predict_batch(lp_queue)  # iterator over predictions

    # function to put detected license plate box into queue
    def queue_license_plate_box(lp_boxes, box_idx):
        lp_img = dgtools.Display.crop(lp_boxes.image, lp_boxes.results[box_idx]["bbox"])
        lp_info = {"lp_box": lp_boxes.results[box_idx]}  # store license plate box info
        if box_idx == len(lp_boxes.results) - 1:  # for last box...
            lp_info["frame"] = lp_boxes  # ...store whole frame
        lp_queue.put((lp_img, lp_info))

    # function to patch license plate box label with recognized license text
    def process_license_plate_text(lp_ocr_result):
        lp_ocr_result.info["lp_box"]["label"] = lp_ocr_result.results[0]["label"]
        if "frame" in lp_ocr_result.info:  # if we have the whole frame...
            display.show(lp_ocr_result.info["frame"])  # display it

    # outer loop over camera frames: apply license plate box detection model
    for lp_boxes in dgtools.predict_stream(lp_det_model, camera_id):
        # put all detected license plate boxes into queue
        for box_idx in range(len(lp_boxes.results)):
            queue_license_plate_box(lp_boxes, box_idx)

        # process all recognized license plates ready so far
        while lp_ocr_result := next(lp_ocr_model_iter):
            process_license_plate_text(lp_ocr_result)

    lp_queue.put(None)  # signal end of queue to nested model
    lp_ocr_model.non_blocking_batch_predict = False  # restore blocking mode
    # process all remaining recognized license plates
    for lp_ocr_result in lp_ocr_model_iter:
        process_license_plate_text(lp_ocr_result)
